# Marina Lebedina
## ID: 12249127

# HW 3. Return Predictions from Trade Flow

# 1. Data Processing

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from tqdm import tnrange
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm

In [2]:
def process_book(file):
    path = "New_data2/{}".format(file) 
    book = pd.read_table(path, sep='|')
    book['received_utc_nanoseconds'] = pd.to_datetime(book['received_utc_nanoseconds'], unit='ns') # GMT
    book['received_utc_nanoseconds'] = book['received_utc_nanoseconds'] - timedelta(hours=5) # Chicago
    book.set_index('received_utc_nanoseconds', inplace=True)
    return book

def process_trades(file):
    path = "New_data2/{}".format(file) 
    trades = pd.read_table(path, sep='|')
    trades['received_utc_nanoseconds'] = pd.to_datetime(trades['received_utc_nanoseconds'], unit='ns') # GMT
    trades['received_utc_nanoseconds'] = trades['received_utc_nanoseconds'] - timedelta(hours=5) # Chicago
    trades['timestamp_utc_nanoseconds'] = pd.to_datetime(trades['timestamp_utc_nanoseconds'], unit='ns') # GMT
    trades['timestamp_utc_nanoseconds'] = trades['timestamp_utc_nanoseconds'] - timedelta(hours=5) # Chicago
    trades.set_index('timestamp_utc_nanoseconds', inplace=True)
    return trades

def subsample(trades, frac):
    threshold =int(frac*trades.shape[0])
    trades_sub = trades[:threshold]
    return trades_sub

def train_test_split(trades, book, ticker, verbose):
    # TRADES
    trades_train_start = trades.index[0]
    trades_train_end = trades.index[int(trades.shape[0]*0.2)] # inclusive
    mask = trades.iloc[abs(trades.index - trades_train_end).argsort()]
    mask = mask[mask.index > trades_train_end]
    trades_test_start = mask.index[0]
    trades_test_end = trades.index[-1]
    # BOOK
    book_train_start = book.index[0]
    book_train_end = book.iloc[abs(book.index - trades_train_end).argsort()[:1]].index[0]
    mask = book.iloc[abs(book.index - book_train_end).argsort()]
    mask = mask[mask.index > book_train_end]
    book_test_start = mask.index[0]
    book_test_end = book.iloc[abs(book.index - trades_test_end).argsort()[:1]].index[0]
    trades_train = trades[trades_train_start:trades_train_end]
    trades_test = trades[trades_test_start:trades_test_end]
    book_train = book[book_train_start:book_train_end]
    book_test = book[book_test_start:book_test_end]
    if verbose == True:
        print('{} TRADES TRAIN START \t= {}'.format(ticker, trades_train_start))
        print('{} TRADES TRAIN END \t= {}'.format(  ticker, trades_train_end))
        print('{} TRADES TEST START \t= {}'.format( ticker, trades_test_start))
        print('{} TRADES TEST END \t= {}\n'.format( ticker, trades_test_end))
        print('{} BOOK TRAIN START \t= {}'.format(ticker, book_train_start))
        print('{} BOOK TRAIN END \t= {}'.format(  ticker, book_train_end))
        print('{} BOOK TEST START \t= {}'.format( ticker, book_test_start))
        print('{} BOOK TEST END \t= {}\n'.format( ticker, book_test_end))
    print('{} TRADES train/test/total length = {} / {} / {}, {}'.format(ticker, trades_train.shape[0], 
        trades_test.shape[0], trades_train.shape[0] + trades_test.shape[0], trades.shape[0]))
    print('{} BOOK train/test/total length = {} / {} / {}, {}'.format(ticker, book_train.shape[0], 
        book_test.shape[0], book_train.shape[0] + book_test.shape[0], book.shape[0]))
    return trades_train, trades_test, book_train, book_test

# 2. Flow calculation and Forward Return calculation / prediction

1. **Flow calculation**

$$ F_t^{(\tau)} = V_{t-\tau, t}^B - V_{t-\tau, t}^S $$

where:

$V^B$ - volume of buyer-initiated trades;

$V^S$ - volume of seller-initiated trades;

$F_t^{(\tau)}$ - flow corresponding to time $t$;

$\tau$ - threshold for flow calculation;

2. **Forward Return calculation**

$$ r_t = \frac{P_{t+T}}{P_t} - 1 $$

where:

$P_t$ - price of the asset at current time stamp $t$;

$P_{t+T}$ - price of the asset $T$ seconds from current time stamp $t$;

$T$ - threshold for forward return calculation;


3. **Regression** (regress forward returns on flows)

$$ r_t = \beta \cdot F_t^{(\tau)} + \alpha $$


4. **Forward Return prediction** (predict future forward returns by multiplying estimated $\beta$ on actual flows)

$$ \hat{r}_i = \beta \cdot F_i^{(\tau)} $$

In [3]:
def flow_calc(trades, tau_sec):
    unique_times = trades.index.unique()
    tau = timedelta(seconds = tau_sec)
    trades['Flow'] = 0.00
    for i in tnrange(len(unique_times), desc='Flow calculation'): 
        if i == 0:
            flow = 0
        else:
            t_actual = unique_times[i] 
            # not including trading time
            mask = trades.iloc[abs(trades.index - t_actual).argsort()]
            mask = mask[mask.index < t_actual]
            t = mask.index[0]
            t_tau_actual = t_actual - tau
            # including t-tau time
            t_tau = trades.iloc[abs(trades.index - t_tau_actual).argsort()[:1]].index[0]
            flow_mask = trades[t_tau:t]
            flow = (-flow_mask['Side']*flow_mask['SizeBillionths']).sum()
            timestamp_pull = trades.iloc[trades.index == t_actual]
            for j in range(len(timestamp_pull.index)):
                date = timestamp_pull.index[j]
                trades.loc[date, 'Flow'] = flow
    return trades

def ret_calc(trades, book, T_sec):
    times = trades.index
    T = timedelta(seconds = T_sec)
    trades['Forw_ret'] = 0.00
    for i in tnrange(len(times), desc='Forward Return calculation'): 
        if i == 0:
            flow = 0
        else:
            t_actual = times[i] 
            t = t_actual
            t_T_actual = t_actual + T
            t_T = book.iloc[abs(book.index - t_T_actual).argsort()[:1]].index[0]
            P_t = trades.loc[t,'PriceMillionths']
            P_tT = book.loc[t_T,'Mid']
            r_t = (P_tT / P_t) - 1
            timestamp_pull = trades.iloc[trades.index == t_actual]
            if isinstance(P_t, pd.Series) == True:
                for j in range(len(timestamp_pull.index)):
                    date = timestamp_pull.index[j]
                    trades.loc[date, 'Forw_ret'] = r_t[j]
            else:
                trades.loc[t_actual, 'Forw_ret'] = r_t
    return trades

def predict(train_ret, test_ret, ticker):
    mult_v = 1000000000
    x = np.array(train_ret['Flow']) / mult_v
    y = np.array(train_ret['Forw_ret'])
    reg = sm.OLS(endog=y, exog=x)
    res = reg.fit()
    beta = res.params[0]
    print('{} beta = {}'.format(ticker, beta))
    r_pred = (np.array(test_ret['Flow']) / mult_v) * beta
    return beta, res.pvalues[0], res.tvalues[0], r_pred

# 3. Trading Strategy Implementation

After calculation of predicted forward returns, we will implement trading strategy which is based on the following trading rule:

$$ r_t > j \Rightarrow \text{ go LONG at time } t \text{ and go SHORT at time } t+T $$
$$ r_t < -j \Rightarrow \text{ go SHORT at time } t \text{ and go LONG at time } t+T $$ 

where: 

$r_t$ - predicted Forward Return, 

$j$ - "trading threshold",

$T$ - "forward return calculation threshold".

All in all **trading strategy** is based on the following idea:
- if predicted forward return corresponding to time $t$ is greater than a pre-specified threshold $j$, we will buy the cryptocurrency at moment $t$ and will sell it in $T$ seconds from moment $t$ to close the position;

- if predicted forward return corresponding to time $t$ is less than a $-j$, we will sell the cryptocurrency at moment $t$ and will buy it in $T$ seconds from moment $t$ to close the position;

In [4]:
def trading(trades_ret, book_test, j, T_sec):
    
    book = book_test.copy()
    df = trades_ret.copy()
    df = df.loc[~df.index.duplicated(keep='first')]
    df.drop(['PriceMillionths', 'Side', 'SizeBillionths', 'received_utc_nanoseconds', 'Flow'], \
                                                                    axis=1, inplace=True)
    df['Open_time'] = ""
    df['Open_$'] = 0.00
    df['Close_time'] = ""
    df['Close_$'] = 0.00
    df['Idea'] = ""
    df['PnL'] = 0.00
    df['Return'] = 0.00
    df['Return_log'] = 0.00
    
    T = timedelta(seconds=T_sec)

    for i in tnrange(df.shape[0], desc='Trading strategy implementation'):

        t_actual = df.index[i]

        if df['r_t'][i] > j:

            df['Idea'][i] = 'BUY'

            t = book.iloc[abs(book.index - t_actual).argsort()[:1]].index[0]
            P_t = book.loc[t, 'Ask1PriceMillionths']
            df['Open_time'][i] = t
            df['Open_$'][i] = P_t

            tT_actual = t + T
            tT = book.iloc[abs(book.index - tT_actual).argsort()[:1]].index[0]
            P_tT = book.loc[tT, 'Bid1PriceMillionths']
            df['Close_time'][i] = tT
            df['Close_$'][i] = P_tT

            df['PnL'][i] = P_tT - P_t
            df['Return'][i] = df['PnL'][i] / df['Open_$'][i]
            df['Return_log'][i] = np.log(df['Return'][i] + 1)

        elif df['r_t'][i] < -j:

            df['Idea'][i] = 'SELL'

            t = book.iloc[abs(book.index - t_actual).argsort()[:1]].index[0]
            P_t = book.loc[t, 'Bid1PriceMillionths']
            df['Open_time'][i] = t
            df['Open_$'][i] = P_t

            tT_actual = t + T
            tT = book.iloc[abs(book.index - tT_actual).argsort()[:1]].index[0]
            P_tT = book.loc[tT, 'Ask1PriceMillionths']
            df['Close_time'][i] = tT
            df['Close_$'][i] = P_tT

            df['PnL'][i] = -(P_tT - P_t)
            df['Return'][i] = df['PnL'][i] / df['Close_$'][i]
            df['Return_log'][i] = np.log(df['Return'][i] + 1)

    df['PnL_cum'] = df['PnL'].cumsum()

    return df

# 4. Parameters tuning

Trading Strategy considered here has several input parameters, i.e. $\tau, T, j$ as well as estimated parameter $\beta$. Thus, the model will be fine tuned by **testing different combinations of parameters in an iterative way** and comparing performance of the model. Performance of strategy will be assessed based on:
- **Cumulative PnL**;

- **Mean Simple Return**;

- **Sharpe Ratio**;

- **Sortino Ratio**. 

**Reliability of the beta $\beta$ coefficient** will be assessed based on its **$p-value$**. 

As we are working with tick data, it is reasonable to consider ***small time intervals*** for flow as well as forward return calculations. That is why we will try the following time intervals as $\tau$ and $T$ values: **5, 10, 30 and 60 seconds**. 
Parameter $j$ which plays role in trading signals generation will also be chosen in iterative way. Possible values for it are selected based on the fact that we ***want to differentiate "noise" from reliable predictions as well as desire not to do too frequent trading***. That's why values for $j$ are selected to be no less than mean of predicted returns plus one standard deviation and are equal to: **0.001, 0.003, 0.005**, i.e. $0.1\%, 0.3\%, 0.5\%$ predicted position returns. 

As mentioned above, we will open a long position at time $t$ if predicted forward return corresponding to time $t$ is greater than the threshold of return $j$, and close the position in $T$ seconds from its opening.  We will open a short position at time $t$ if predicted forward return corresponding to time $t$ is less than the negative of threshold of return $-j$, and close the position in $T$ seconds from its opening.

Best set of parameters ($\tau, T, j)$ will be chosen based on the value of Sharpe and Sortiono Ratios as well as Cumulative PnL. The higher the ratios and PnL, the better the model is from the perspective of both risk and return.

We will test reliability of beta $\beta$ coefficient based on its $p-value$ using $95\%$-confidence level, i.e.:

- if $p-value(\beta) < 0.05 \Rightarrow \beta \text{ can be considered to be a reliable coefficient estimate}$;

- if $p-value(\beta) > 0.05 \Rightarrow \beta \text{ can be considered to be NOT a reliable coefficient estimate}$. 

We will test trading strategy on three cryptocurrency pairs: BTC-USD, ETH-USD, BTC-ETH.

First we will work with a **subsample of the whole available datasets in order to test a wide range of parameter combinations** and choose the best one. Afterwards we will **run trading strategy using the best set of parameters on the whole dataset** in order to test the strategy on a longer horizon as well as to answer the question what can we expect in general from using much longer training and test periods and whether they contribute to more reliable betas and better strategy performance. 

In [5]:
def tuning(frac, tau, T, j, file_b, file_t, ticker):
    
    trades = process_trades(file_t)
    book = process_book(file_b)

    trades = subsample(trades, frac)

    trades_train, trades_test, book_train, book_test = train_test_split(trades, book, ticker, False)

    trades_train_flow = flow_calc(trades_train, tau) 
    trades_test_flow = flow_calc(trades_test, tau)

    trades_train_ret = ret_calc(trades_train_flow, book_train, T)  

    beta, pv, tv, pred = predict(trades_train_ret, trades_test_flow, ticker)
    trades_test_flow['r_t'] = pred

    tr_df = trading(trades_test_flow, book_test, j, T)
    
    if tr_df['Return'].std() != 0.00:
        SR_mod = (tr_df['Return'].mean() - 0) / tr_df['Return'].std()
    else:
        SR_mod = 0.00
    
    mask_neg_r = (tr_df['Return'] < 0) 
    neg_r = tr_df[mask_neg_r]
    if neg_r['Return'].std() != 0.00:
        SorR = (tr_df['Return'].mean() - 0) / neg_r['Return'].std()
    else:
        SorR = 0.00
    
    return tr_df, tr_df['Return'].mean(), SR_mod, SorR, beta, pv, tv

In [6]:
taus = [5,10,30,60]
Ts = [5,10,30,60]
js = [0.001,0.003,0.005]

indices = []
for i in range(len(taus)):
    for j in range(len(Ts)):
        for k in range(len(js)):
            idx = 'tau={},T={},j={}'.format(taus[i],Ts[j],js[k])
            indices.append(idx)

## 4.1. Parameter tuning for $BTC-USD$

In [7]:
btc_stat = pd.DataFrame(index = indices, \
            columns=['PnL_cumulative', 'Return', 'Sharpe_Ratio_modified', \
                     'Sortino_Ratio', 'Beta', 'p-v', 't-v'])

file_b = "book_narrow_BTC-USD_2018.delim"
file_t = "trades_narrow_BTC-USD_2018.delim"
ticker = 'BTC_USD'
frac = 0.01

for i in range(len(taus)): 
    tau = taus[i]
    for j in range(len(Ts)): 
        T = Ts[j]
        for k in range(len(js)): 
            j = js[k]
            idx = 'tau={},T={},j={}'.format(tau,T,j)
            res, ret, SR, SorR, beta, pv, tv = tuning(frac, tau, T, j, file_b, file_t, ticker)
            btc_stat['PnL_cumulative'][idx] = res['PnL_cum'][-1]/1000000
            btc_stat['Return'][idx] = ret
            btc_stat['Sharpe_Ratio_modified'][idx] = SR
            btc_stat['Sortino_Ratio'][idx] = SorR
            btc_stat['Beta'][idx] = beta
            btc_stat['p-v'][idx] = pv
            btc_stat['t-v'][idx] = tv

BTC_USD TRADES train/test/total length = 1160 / 4638 / 5798, 5798
BTC_USD BOOK train/test/total length = 88304 / 33898 / 122202, 3756140



BTC_USD beta = 0.0005316581578554147



BTC_USD TRADES train/test/total length = 1160 / 4638 / 5798, 5798
BTC_USD BOOK train/test/total length = 88304 / 33898 / 122202, 3756140



BTC_USD beta = 0.0005316581578554147



BTC_USD TRADES train/test/total length = 1160 / 4638 / 5798, 5798
BTC_USD BOOK train/test/total length = 88304 / 33898 / 122202, 3756140


...


BTC_USD beta = 0.0003308389317521514



BTC_USD TRADES train/test/total length = 1160 / 4638 / 5798, 5798
BTC_USD BOOK train/test/total length = 88304 / 33898 / 122202, 3756140



BTC_USD beta = 0.0003308389317521514



BTC_USD TRADES train/test/total length = 1160 / 4638 / 5798, 5798
BTC_USD BOOK train/test/total length = 88304 / 33898 / 122202, 3756140



BTC_USD beta = 0.0003308389317521514


## 4.2. Parameters tuning for $ETH-USD$

In [8]:
eth_stat = pd.DataFrame(index = indices, \
            columns=['PnL_cumulative', 'Return', 'Sharpe_Ratio_modified', 'Sortino_Ratio', 'Beta', 'p-v', 't-v'])

file_b = 'book_narrow_ETH-USD_2018.delim'
file_t = 'trades_narrow_ETH-USD_2018.delim'
ticker = 'ETH_USD'
frac = 0.01

for i in range(len(taus)): 
    tau = taus[i]
    for j in range(len(Ts)): 
        T = Ts[j]
        for k in range(len(js)): 
            j = js[k]
            idx = 'tau={},T={},j={}'.format(tau,T,j)
            res, ret, SR, SorR, beta, pv, tv = tuning(frac, tau, T, j, file_b, file_t, ticker)
            eth_stat['PnL_cumulative'][idx] = res['PnL_cum'][-1]/1000000
            eth_stat['Return'][idx] = ret
            eth_stat['Sharpe_Ratio_modified'][idx] = SR
            eth_stat['Sortino_Ratio'][idx] = SorR
            eth_stat['Beta'][idx] = beta
            eth_stat['p-v'][idx] = pv
            eth_stat['t-v'][idx] = tv

ETH_USD TRADES train/test/total length = 815 / 3051 / 3866, 3866
ETH_USD BOOK train/test/total length = 39573 / 23169 / 62742, 2093729



ETH_USD beta = -0.00034282664006384455



ETH_USD TRADES train/test/total length = 815 / 3051 / 3866, 3866
ETH_USD BOOK train/test/total length = 39573 / 23169 / 62742, 2093729



ETH_USD beta = -0.00034282664006384455



ETH_USD TRADES train/test/total length = 815 / 3051 / 3866, 3866
ETH_USD BOOK train/test/total length = 39573 / 23169 / 62742, 2093729


...


ETH_USD beta = -0.0001054826937210109



ETH_USD TRADES train/test/total length = 815 / 3051 / 3866, 3866
ETH_USD BOOK train/test/total length = 39573 / 23169 / 62742, 2093729



ETH_USD beta = -0.0001054826937210109



ETH_USD TRADES train/test/total length = 815 / 3051 / 3866, 3866
ETH_USD BOOK train/test/total length = 39573 / 23169 / 62742, 2093729



ETH_USD beta = -0.0001054826937210109


## 4.3. Parameters tuning for $ETH-BTC$

In [9]:
eth_btc_stat = pd.DataFrame(index = indices, \
            columns=['PnL_cumulative', 'Return', 'Sharpe_Ratio_modified', 'Sortino_Ratio', 'Beta', 'p-v', 't-v'])

file_b = 'book_narrow_ETH-BTC_2018.delim'
file_t = 'trades_narrow_ETH-BTC_2018.delim'
ticker = 'ETH_BTC'
frac = 0.01

for i in range(len(taus)): 
    tau = taus[i]
    for j in range(len(Ts)): 
        T = Ts[j]
        for k in range(len(js)): 
            j = js[k]
            idx = 'tau={},T={},j={}'.format(tau,T,j)
            res, ret, SR, SorR, beta, pv, tv = tuning(frac, tau, T, j, file_b, file_t, ticker)
            eth_btc_stat['PnL_cumulative'][idx] = res['PnL_cum'][-1]/1000000
            eth_btc_stat['Return'][idx] = ret
            eth_btc_stat['Sharpe_Ratio_modified'][idx] = SR
            eth_btc_stat['Sortino_Ratio'][idx] = SorR
            eth_btc_stat['Beta'][idx] = beta
            eth_btc_stat['p-v'][idx] = pv
            eth_btc_stat['t-v'][idx] = tv

ETH_BTC TRADES train/test/total length = 61 / 240 / 301, 301
ETH_BTC BOOK train/test/total length = 9415 / 5310 / 14725, 423955



ETH_BTC beta = 0.0010540643630959335



ETH_BTC TRADES train/test/total length = 61 / 240 / 301, 301
ETH_BTC BOOK train/test/total length = 9415 / 5310 / 14725, 423955



ETH_BTC beta = 0.0010540643630959335



ETH_BTC TRADES train/test/total length = 61 / 240 / 301, 301
ETH_BTC BOOK train/test/total length = 9415 / 5310 / 14725, 423955


...


ETH_BTC beta = 0.0006792597954595822



ETH_BTC TRADES train/test/total length = 61 / 240 / 301, 301
ETH_BTC BOOK train/test/total length = 9415 / 5310 / 14725, 423955



ETH_BTC beta = 0.0006792597954595822



ETH_BTC TRADES train/test/total length = 61 / 240 / 301, 301
ETH_BTC BOOK train/test/total length = 9415 / 5310 / 14725, 423955



ETH_BTC beta = 0.0006792597954595822


## 4.4. Analysis of parameters and strategy performance on subsamples

**ETH-USD:**

We can see from the table below that trading strategy shows the best results when we use the following parameters: $\tau = 30, T = 60$ seconds and $j = 0.003$:

- **Cumulative PnL = 25.32**;

- **Sharpe Ratio modified = 0.105** with zero as a benchmark for its calculation (i.e. $r_b = 0$). It tells us that performance of our trading strategy taking into account the OVERALL risk of our strategy is above the benchmark, and we have positive excess returns, i.e. in general we can just say that we have positive returns as benchmark is $0$. 

- **Sortino Ratio = 0.069** with zero as a benchmark for its calculation (i.e. $r_b = 0$). It tells us that performance of our trading strategy taking into account ONLY the DOWNSIDE RISK of our strategy (measured by standard deviation of positions with negative returns) is above zero, and we have positive returns adjusted for downside risk. 

- **Beta = 0.00011** and tells us that there is a positive relationahip between Flow and Forward returns, and in particular, forward retuns increase by 0.00011 (0.011%) for every 1 bn increase in the Flow. $p-value(\beta) = 1.9516e-83 < 0.05 \Rightarrow \beta $ seems to be a realiable eatimate. 

Taking into consideration common benchmark values for Sharpe and Sortino ratios and the fact that those ratios for our trading strategy are less than 1, we can say that returns delivered by this trading strategy on ETH-USD pair do not make up for the amount of underlying risk.

In [10]:
# Check parameters corresponding to the highest cumulative PnL
eth_stat[eth_stat['PnL_cumulative'] == eth_stat['PnL_cumulative'].max()]

,PnL_cumulative,Return,Sharpe_Ratio_modified,Sortino_Ratio,Beta,p-v,t-v
"tau=30,T=60,j=0.003",25.32,7.73221e-05,0.105263,0.0685712,-0.000112251,1.9516e-83,-21.8151


In [11]:
# Check parameters corresponding to the highest Return
eth_stat[eth_stat['Return'] == eth_stat['Return'].max()]

,PnL_cumulative,Return,Sharpe_Ratio_modified,Sortino_Ratio,Beta,p-v,t-v
"tau=30,T=60,j=0.003",25.32,7.73221e-05,0.105263,0.0685712,-0.000112251,1.9516e-83,-21.8151


In [12]:
# Check parameters corresponding to the highest modified Sharpe Ratio
eth_stat[eth_stat['Sharpe_Ratio_modified'] == eth_stat['Sharpe_Ratio_modified'].max()]

,PnL_cumulative,Return,Sharpe_Ratio_modified,Sortino_Ratio,Beta,p-v,t-v
"tau=30,T=60,j=0.005",20.93,6.37887e-05,0.110811,0.0565547,-0.000112251,1.9516e-83,-21.8151


In [13]:
# Check parameters corresponding to the highest Sortino Ratio
eth_stat[eth_stat['Sortino_Ratio'] == eth_stat['Sortino_Ratio'].max()]

,PnL_cumulative,Return,Sharpe_Ratio_modified,Sortino_Ratio,Beta,p-v,t-v
"tau=30,T=60,j=0.003",25.32,7.73221e-05,0.105263,0.0685712,-0.000112251,1.9516e-83,-21.8151


**ETH-BTC:**

We can see from the table below that trading strategy when applied to ETH-BTC cryptocurrency pair shows the best results when we use the following parameters: $\tau = 60, T = 60$ seconds and $j = 0.005$:

- **Cumulative PnL = \$-9e-05** $\approx$ **\$ 0.00**;

- **Sharpe Ratio modified = -0.15** with zero as a benchmark for its calculation (i.e. $r_b = 0$). It tells us that performance of our trading strategy taking into account the OVERALL risk of our strategy is below the benchmark, and we have negative excess returns, i.e. in general we can just say that we have negative returns as benchmark is $0$. 

- **Sortino Ratio = -0.040** with zero as a benchmark for its calculation (i.e. $r_b = 0$). It tells us that performance of our trading strategy taking into account ONLY the DOWNSIDE RISK of our strategy (measured by standard deviation of positions with negative returns) is below zero, and we have negative returns adjusted for downside risk. As Sortino Ratio is higher than Sharpe Ratio we can say that trading stretegy performs better in downturns than in upturns taking into consideration underlying risk. 

- **Beta = -0.00068** and tells us that there is a negative relationahip between Flow and Forward returns, and in particular, forward retuns decrease by 0.00068 (0.068\%) for every 1 bn increase in the Flow. $p-value(\beta) = 6.17154e-05 < 0.05 \Rightarrow \beta $ seems to be a realiable eatimate. 

In [14]:
# Check parameters corresponding to the highest cumulative PnL
eth_btc_stat[eth_btc_stat['PnL_cumulative'] == eth_btc_stat['PnL_cumulative'].max()]

,PnL_cumulative,Return,Sharpe_Ratio_modified,Sortino_Ratio,Beta,p-v,t-v
"tau=60,T=60,j=0.005",-9e-05,-1.25882e-05,-0.14971,-0.0398182,0.00067926,6.17154e-05,4.30956


In [15]:
# Check parameters corresponding to the highest Return
eth_btc_stat[eth_stat['Return'] == eth_btc_stat['Return'].max()]

,PnL_cumulative,Return,Sharpe_Ratio_modified,Sortino_Ratio,Beta,p-v,t-v


In [16]:
# Check parameters corresponding to the highest modified Sharpe Ratio
eth_btc_stat[eth_btc_stat['Sharpe_Ratio_modified'] == eth_btc_stat['Sharpe_Ratio_modified'].max()]

,PnL_cumulative,Return,Sharpe_Ratio_modified,Sortino_Ratio,Beta,p-v,t-v
"tau=60,T=60,j=0.005",-9e-05,-1.25882e-05,-0.14971,-0.0398182,0.00067926,6.17154e-05,4.30956


In [17]:
# Check parameters corresponding to the highest Sortino Ratio
eth_btc_stat[eth_btc_stat['Sortino_Ratio'] == eth_btc_stat['Sortino_Ratio'].max()]

,PnL_cumulative,Return,Sharpe_Ratio_modified,Sortino_Ratio,Beta,p-v,t-v
"tau=60,T=60,j=0.005",-9e-05,-1.25882e-05,-0.14971,-0.0398182,0.00067926,6.17154e-05,4.30956


**BTC-USD:**

We can see from the table below that trading strategy when applied to BTC-USD cryptocurrency pair shows the best results when we use the following parameters: $\tau = 10, T = 10$ seconds and $j = 0.005$:

- **Cumulative PnL = \$-23.31**;

- **Sharpe Ratio modified = -0.011** with zero as a benchmark for its calculation (i.e. $r_b = 0$). It tells us that performance of our trading strategy taking into account the OVERALL risk of our strategy is below the benchmark, and we have negative excess returns, i.e. in general we can just say that we have negative returns as benchmark is $0$. 

- **Sortino Ratio = -0.0040** with zero as a benchmark for its calculation (i.e. $r_b = 0$). It tells us that performance of our trading strategy taking into account ONLY the DOWNSIDE RISK of our strategy (measured by standard deviation of positions with negative returns) is below zero, and we have negative returns adjusted for downside risk. As Sortino Ratio is higher than Sharpe Ratio we can say that trading stretegy performs better in downturns than in upturns taking into consideration underlying risk. 

- **Beta = 0.00051** and tells us that there is a positive relationahip between Flow and Forward returns, and in particular, forward retuns increase by 0.00051 (0.051\%) for every 1 bn increase in the Flow. $p-value(\beta) = 1.55887e-11 < 0.05 \Rightarrow \beta $ seems to be a realiable eatimate. 

In [18]:
# Check parameters corresponding to the highest cumulative PnL
btc_stat[btc_stat['PnL_cumulative'] == btc_stat['PnL_cumulative'].max()]

,PnL_cumulative,Return,Sharpe_Ratio_modified,Sortino_Ratio,Beta,p-v,t-v
"tau=10,T=10,j=0.005",-23.31,-2.16255e-06,-0.0107139,-0.00398503,0.000505767,1.55887e-11,6.81046


In [19]:
# Check parameters corresponding to the highest Return
btc_stat[eth_stat['Return'] == btc_stat['Return'].max()]

,PnL_cumulative,Return,Sharpe_Ratio_modified,Sortino_Ratio,Beta,p-v,t-v


In [20]:
# Check parameters corresponding to the highest modified Sharpe Ratio
btc_stat[btc_stat['Sharpe_Ratio_modified'] == btc_stat['Sharpe_Ratio_modified'].max()]

,PnL_cumulative,Return,Sharpe_Ratio_modified,Sortino_Ratio,Beta,p-v,t-v
"tau=10,T=10,j=0.005",-23.31,-2.16255e-06,-0.0107139,-0.00398503,0.000505767,1.55887e-11,6.81046


In [21]:
# Check parameters corresponding to the highest Sortino Ratio
btc_stat[btc_stat['Sortino_Ratio'] == btc_stat['Sortino_Ratio'].max()]

,PnL_cumulative,Return,Sharpe_Ratio_modified,Sortino_Ratio,Beta,p-v,t-v
"tau=10,T=10,j=0.005",-23.31,-2.16255e-06,-0.0107139,-0.00398503,0.000505767,1.55887e-11,6.81046


# 5. Analysis of strategy performance on the full dataset using the "best" set of parameters

## 5.1. ETH-USD
#### $tau=30, T=60, j=0.003$

In [23]:
indices_fs = ['ETH_USD: tau=30,T=60,j=0.003', 'ETH_BTC: tau=60,T=60,j=0.005', 'BTC_USD: tau=10,T=10,j=0.005']
full_stat = pd.DataFrame(index = indices_fs, \
            columns=['PnL_cumulative', 'Return', 'Sharpe_Ratio_modified', 'Sortino_Ratio', 'Beta', 'p-v', 't-v'])

file_b = 'book_narrow_ETH-USD_2018.delim'
file_t = 'trades_narrow_ETH-USD_2018.delim'
ticker = 'ETH_USD'
frac = 1

tau = 30
T = 60
j = 0.003

idx = '{}: tau={},T={},j={}'.format(ticker,tau,T,j)
res, ret, SR, SorR, beta, tv, pv = tuning(frac, tau, T, j, file_b, file_t, ticker)

ETH_USD TRADES train/test/total length = 38669 / 154671 / 193340, 193340
ETH_USD BOOK train/test/total length = 245515 / 869744 / 1115259, 2093729



ETH_USD beta = 1.5140118576346703e-06


In [24]:
full_stat['PnL_cumulative'][idx] = res['PnL_cum'][-1]/1000000
full_stat['Return'][idx] = ret
full_stat['Sharpe_Ratio_modified'][idx] = SR
full_stat['Sortino_Ratio'][idx] = SorR
full_stat['Beta'][idx] = beta
full_stat['p-v'][idx] = pv
full_stat['t-v'][idx] = tv

## 5.2. ETH-BTC
#### $tau=60, T=60, j=0.005$

In [29]:
file_b = 'book_narrow_ETH-BTC_2018.delim'
file_t = 'trades_narrow_ETH-BTC_2018.delim'
ticker = 'ETH_BTC'
frac = 1

tau = 60
T = 60
j = 0.005

idx = '{}: tau={},T={},j={}'.format(ticker,tau,T,j)
res, ret, SR, SorR, beta, tv, pv = tuning(frac, tau, T, j, file_b, file_t, ticker)

ETH_BTC TRADES train/test/total length = 6039 / 24151 / 30190, 30190
ETH_BTC BOOK train/test/total length = 85842 / 338113 / 423955, 423955



ETH_BTC beta = -5.777553337725149e-07


In [30]:
full_stat['PnL_cumulative'][idx] = res['PnL_cum'][-1]/1000000
full_stat['Return'][idx] = ret
full_stat['Sharpe_Ratio_modified'][idx] = SR
full_stat['Sortino_Ratio'][idx] = SorR
full_stat['Beta'][idx] = beta
full_stat['p-v'][idx] = pv
full_stat['t-v'][idx] = tv

## 5.3. BTC-USD
#### $tau=10, T=10, j=0.005$

In [33]:
file_b = 'book_narrow_BTC-USD_2018.delim'
file_t = 'trades_narrow_BTC-USD_2018.delim'
ticker = 'BTC_USD'
frac = 1

tau = 10
T = 10
j = 0.005

idx = '{}: tau={},T={},j={}'.format(ticker,tau,T,j)
res, ret, SR, SorR, beta, tv, pv = tuning(frac, tau, T, j, file_b, file_t, ticker)

ETH_BTC TRADES train/test/total length = 57983 / 231930 / 289913, 289913
ETH_BTC BOOK train/test/total length = 442053 / 1423163 / 1865216, 3756140



ETH_BTC beta = 9.438837744701581e-05


In [46]:
full_stat['PnL_cumulative'][idx] = res['PnL_cum'][-1]/1000000
full_stat['Return'][idx] = ret
full_stat['Sharpe_Ratio_modified'][idx] = SR
full_stat['Sortino_Ratio'][idx] = SorR
full_stat['Beta'][idx] = beta
full_stat['p-v'][idx] = pv
full_stat['t-v'][idx] = tv

In [69]:
full_stat

,PnL_cumulative,Return,Sharpe_Ratio_modified,Sortino_Ratio,Beta,p-v,t-v
"ETH_USD: tau=30,T=60,j=0.003",388.13,1.78093e-05,0.0634291,0.0095429,1.51401e-06,0.164123,1.39136
"ETH_BTC: tau=60,T=60,j=0.005",0,0,0,0,-5.77755e-07,0.740589,-0.331088
"BTC_USD: tau=10,T=10,j=0.005",-187.8,-3.37921e-07,-0.0115386,-0.00150572,9.43884e-05,6.30588e-71,17.8308


As we can see from the summary table above, trading strategy does not show a better performance in terms of Sharpe and Sortino ratios as well as in terms of cumulative PnL and Simple Return when we use longer training and test periods. Moreover reliability of beta estimate decreases for ETH-USD and ETH-BTC pairs when we use longer horizon as their $p-values(\beta) > 0.05$. 

Based on the above analysis we can conclude that trading strategy does not perform well on tick data of cryptocurrency pairs BTC and ETH over a weekly as well as daily investment horizon. Moreover, in the investment period longer than 1 day reliability of prediction of forward returns drops dramatically, which can be explained by the fact that there are other factors than the Flow which determine behavior of cryptocurrency over a longer than 1-day horizon, and thus usage of trading strategy which is solely based on analysis of Flow is not justified.